## Carga de librerías

In [31]:
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import gc
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

import time
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.externals import joblib
from sklearn.semi_supervised import label_propagation

## lectura de datos
Se carga el dataset que contiene los datos depurados y listos para ser entrenados

In [2]:
Consolidado=pd.read_csv('Dataset_consolidado.csv', error_bad_lines=False,warn_bad_lines=False,low_memory=False)

#### Visualización del dataset

In [3]:
Consolidado.head(5)

,seg_str_EMPRENDEDOR,seg_str_OTRO,seg_str_PERSONAL,seg_str_PERSONAL PLUS,seg_str_PREFERENCIAL,ocupacion_1,ocupacion_2,ocupacion_3,ocupacion_4,ocupacion_5,...,nivel_academico_I,nivel_academico_N,nivel_academico_P,nivel_academico_S,nivel_academico_T,nivel_academico_U,fecha,hora,valor_trx,etiqueta
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,20171009.0,90609.0,15100.91,1.0
1,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20171013.0,101444.0,157299.02,2.0
2,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20170825.0,175028.0,371718.55,2.0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20180124.0,104113.0,382742.66,2.0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,20180325.0,153255.0,125253.38,2.0


### Misma cantidad de registros por etiqueta 

se escogen la misma cantidad de registros por etiqueta para que el dataset quede balanceado.

se utiliza la funcion shuffle con el fin de tomar distintos registros cada vez que se ejecuta el notebook.

In [4]:
Consolidado=pd.concat([shuffle(Consolidado[Consolidado.etiqueta==1]).iloc[0:35857],shuffle(Consolidado[Consolidado.etiqueta==2]).iloc[0:35857],shuffle(Consolidado[Consolidado.etiqueta==3])])

#### Asignación de la variable respuesta

In [5]:
y=Consolidado.etiqueta

#### Asignación de las variables predictoras

In [6]:
X=Consolidado.iloc[:, Consolidado.columns != 'etiqueta']

#### Se libera la memoria

In [7]:
del Consolidado

In [8]:
gc.collect()

7

#### Partición de los datos en entrenamiento y prueba

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.90,stratify=y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.25,stratify=y_train)

In [21]:
np.unique(y_train2)

array([1., 2., 3.])

In [22]:
X_train2.shape[0]

8067

#### Reemplazo de etiquetas
Se Toma una porción de los datos etiquetados para ser marcados sin etiqueta y entrenar el modelo como semisupervisado con etiquetas faltantes.

In [23]:
rng = np.random.RandomState(0)
y_train2[rng.rand(len(y_train2)) < 0.2] = -1 # se quita el 20% de las etiquetas

In [24]:
np.unique(y_train2)

array([-1.,  1.,  2.,  3.])

## Definición del modelo
Al dataset se le incorpora una reducción de dimensionalidad, luego se implementa el modelo label_propagation este se basa en definir fronteras de datos etiquetados y tras cada iteración redefine las fronteras basandose en la distancia de los nuevos datos, sin importar que estos datos no tengan una etiqueta definida. Finalmente se aplica GridSearchCV, el cual basado en validación cruzada y el score balanced_accuracy encuentra el mejor modelo dado por los diferentes hiperparametros.

Se elige balanced_accuracy como medición del score para modelos multiclase desbalanceados.


In [25]:
parametros = {"pca__n_components": [3,5,7],
              "l_propagation__alpha": [0.2],
              "l_propagation__gamma": [15],
              "l_propagation__max_iter": [30],
              "l_propagation__n_neighbors": [5]}

pipeline = Pipeline(steps=[('pca', PCA()),
                           
                           ('l_propagation', label_propagation.LabelSpreading())])


score='balanced_accuracy'

grid_search_LP=GridSearchCV(pipeline, param_grid=parametros,cv=10,scoring=score,n_jobs=-1)

#### Ajuste del modelo

In [26]:
tiempo_inicio=time.clock()
grid_search_LP.fit(X_train2,y_train2)
tiempo_final=time.clock()
print("tiempo de ejecucion : "+str(int(tiempo_final-tiempo_inicio))+" segundos")

C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


tiempo de ejecucion : 158 segundos


C:\Users\lssro\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


#### Mejores parámetros encontrados 
con los que se construye el modelo

In [27]:
grid_search_LP.best_params_

{'l_propagation__alpha': 0.2,
 'l_propagation__gamma': 15,
 'l_propagation__max_iter': 30,
 'l_propagation__n_neighbors': 5,
 'pca__n_components': 3}

#### Selección del mejor modelo
Según los hiperparametros encontrados

In [28]:
grid_search_LP.best_estimator_

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('l_propagation', LabelSpreading(alpha=0.2, gamma=15, kernel='rbf', max_iter=30, n_jobs=None,
        n_neighbors=5, tol=0.001))])

#### Score del mejor modelo

In [29]:
grid_search_LP.best_score_

0.25

In [30]:
confusion_matrix(y_test2, grid_search_LP.best_estimator_.predict(X_test2))

array([[897,   0,   0],
       [897,   0,   0],
       [896,   0,   0]], dtype=int64)